<a href="https://colab.research.google.com/github/meet7515/CE888/blob/main/lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, plot_confusion_matrix, SCORERS
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
df_raw = pd.read_csv('https://github.com/albanda/CE888/raw/master/lab3/bank-additional-full.csv', delimiter = ';')
df = df_raw.drop(['duration'], axis=1)

In [ ]:

#check for empty rows
print("Number of rows before dropping NaNs: %d" % len(df))
df = df.dropna()
print("Number of rows after dropping NaNs: %d" % len(df))

Number of rows before dropping NaNs: 41188
Number of rows after dropping NaNs: 41188

In [ ]:
features = [col for col in df.columns[:-1]] # each column except the last one
outcomes_of_interest = ['y']
cat_features = [col for col in df.columns[1:10]] + ['poutcome']

In [ ]:
df[outcomes_of_interest] = df[outcomes_of_interest].apply(LabelEncoder().fit_transform)

In [ ]:
X_df = df[features].copy()
y_df = df[outcomes_of_interest].copy()
X_df = pd.get_dummies(X_df, columns=cat_features)

In [2]:
X = X_df.values
y = y_df.values.T[0]

y_df.value_counts().plot(kind='bar')

NameError: ignored

In [ ]:
def imbalanceratio(negative, positive):
    return negative/positive

In [ ]:
imbalanceratio(y_df.value_counts()[1], y_df.value_counts()[0])

0.126956331399803

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50, stratify=y)
X_train.shape, X_test.shape

((32950, 62), (8238, 62))

In [ ]:
y_train.shape, y_test.shape

In [ ]:
dummy_clf = DummyClassifier(strategy='stratified')
dummy_clf.fit(X_train, y_train)
print("Dummy F1 Score: %.2f" % f1_score(y_test, dummy_clf.predict(X_test)))

Dummy F1 Score: 0.10

In [ ]:
SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted'])

In [ ]:
def cvscore(estimator, depth, state=50, folds=10):
    cv = cross_val_score(RandomForestClassifier(n_estimators=estimator, max_depth=depth, random_state=state), X_train, y_train, cv=folds, scoring='f1')
    print(("F1 Score: %.2f" % (cv.mean())))

In [ ]:
cvscore(100, 3)

F1 Score: 0.26

In [ ]:
# Create the parameter grid
param_grid = {
     'n_estimators': [50, 75],
     'max_depth': [3, None],
     'min_samples_split': [2, 3],
     'min_samples_leaf': [1, 2],
     'max_features': [3, 'auto'],
}
# Create a base model
rf = RandomForestClassifier(random_state=50, class_weight='balanced')
# Instantiate the grid search model
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=10, scoring='f1', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)


# Create the parameter grid
param_grid = {
     'n_estimators': [50, 75],
     'max_depth': [3, None],
     'min_samples_split': [2, 3],
     'min_samples_leaf': [1, 2],
     'max_features': [3, 'auto'],
}
# Create a base model
rf = RandomForestClassifier(random_state=50, class_weight='balanced')
# Instantiate the grid search model
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=10, scoring='f1', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)
Fitting 10 folds for each of 32 candidates, totalling 320 fits
GridSearchCV(cv=10,
             estimator=RandomForestClassifier(class_weight='balanced',
                                              random_state=50),
             n_jobs=-1,
             param_grid={'max_depth': [3, None], 'max_features': [3, 'auto'],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 3],
                         'n_estimators': [50, 75]},
             scoring='f1', verbose=2)

In [ ]:
grid_search.best_params_

In [ ]:
df_results = pd.DataFrame(grid_search.cv_results_)

In [ ]:
grid_search.best_score_

0.4958800337866209

In [ ]:
print("F1 Score: %.2f" % f1_score(y_test, grid_search.predict(X_test)))

F1 Score: 0.50

In [ ]:
def top_x_features(x):
    for i in range(x):
        print(X_df.columns[[sorted(importance_sorted, reverse=True)[i][1]]])

In [ ]:
def genenralizationerror(classifier):
    y_bias = (np.nanmean(y_test) - np.nanmean(classifier.predict(X_test), axis=0)) **2
    y_error = np.nanmean((y_test - classifier.predict(X_test)) **2, axis=0)#.mean()
    y_var = np.nanvar(classifier.predict(X_test), axis=0, ddof = 1)
    print(np.nanmean(y_bias) +  np.nanmean(y_var))
    clf_type = "Decision tree"
    print("{0}: {1:.4f} (error) = {2:.4f} (bias^2) "
              "+ {3:.4f} (var)".format(clf_type, np.nanmean(y_error), np.nanmean(y_bias), np.nanmean(y_var)))
    print("{0}: {1:.4f} ((bias^2) + (var)) = {2:.4f} (bias^2) "
              "+ {3:.4f} (var)".format(clf_type, np.mean(y_bias) + np.mean(y_var), np.mean(y_bias), np.mean(y_var)))

In [ ]:
importance = grid_search.best_estimator_.feature_importances_
importance_sorted = []
for i,v in enumerate(importance):
    importance_sorted.append((v,i))
# plot feature importance
plt.bar([x for x in range(len(importance))], importance)
plt.show()
sorted(importance_sorted, reverse=True)

In [ ]:
top_x_features(3)

Index(['euribor3m'], dtype='object')
Index(['age'], dtype='object')
Index(['nr.employed'], dtype='object')

In [ ]:
genenralizationerror(grid_search)

0.12254337502065248
Decision tree: 0.1277 (error) = 0.0008 (bias^2) + 0.1217 (var)
Decision tree: 0.1225 ((bias^2) + (var)) = 0.0008 (bias^2) + 0.1217 (var)

In [ ]:
plot_confusion_matrix(grid_search, X, y, cmap=plt.cm.Blues)
plt.show()